# 13.LSTM을 이용한 로이터 뉴스 카테고리 분류

### 입력된 문장의 의미를 파악하는 것은 모든 단어를 종합하여 하나의 카테고리로 분류하는 작업
- "안녕, 오늘 날씨가 참 좋아" ==> '인사' 카테고리
- "중부 지방은 대체로 맑겠으나, 남부 지방은 구름이 많겠습니다." ==> 날씨
- "올 초부터 유동성의 힘으로 주가가 일정하게 상승했습니다." ==> 주식
- "퍼셉트론의 한계를 극복한 신경망이 다시 뜨고 있습니다." ==> 딥러닝

### 로이터 뉴스 데이터
* 총 11,258 개의 뉴스 기사
* 46개 카테고리
* MNIST처럼 케라스에서 직접 불러올 수 있음

In [ ]:
# 로이터 뉴스 데이터셋 불러오기
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing import sequence
from keras.utils import np_utils

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# reuters.load_data 실행 시 allow_pickle 관련 에러 해결
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# 불러온 데이터를 학습셋, 테스트셋으로 나누기
(X_train, Y_train), (X_test, Y_test) = reuters.load_data(num_words=1000, test_split=0.2)

# restore np.load for future normal usage
np.load = np_load_old

### 로이터 뉴스 데이터 구조 확인

In [ ]:
category = numpy.max(Y_train) + 1
print(category, '카테고리')
print(len(X_train), '학습용 뉴스 기사')
print(len(X_test), '테스트용 뉴스 기사')
print(X_train[0])

### 데이터 전처리
 기사의 단어 수가 다르므로 단어의 숫자를 맞추기 위한 코드

In [ ]:
len(X_train[0]), len(X_train[1]), len(X_train[2])

In [ ]:
x_train = sequence.pad_sequences(X_train, maxlen=100)
x_test = sequence.pad_sequences(X_test, maxlen=100)

y_train = np_utils.to_categorical(Y_train)
y_test = np_utils.to_categorical(Y_test)

In [ ]:
len(x_train[0]), len(x_train[1]), len(x_train[2])

### Embedding 층
* 입력층의 값을 다음 층이 알아들을 수 있는 형태로 변환
* Embedding(불러온 단어의 총 개수, 기사당 단어 수)

In [ ]:
# 모델의 설정
model = Sequential()
model.add(Embedding(1000, 100))
model.add(LSTM(100, activation='tanh'))
model.add(Dense(46, activation='softmax'))

# 모델의 컴파일
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 모델의 실행
history = model.fit(x_train, y_train, batch_size=100, epochs=20, validation_split=0.2)

### 테스트 데이터 셋으로 모델의 성능 평가

In [ ]:
# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(x_test, y_test)[1]))

### 결과 검토

In [ ]:
# 검증 셋의 오차와 정확도
y_vloss = history.history['val_loss']
y_vacc = history.history['val_acc']

# 학습셋의 오차와 정확도
y_loss = history.history['loss']
y_acc = history.history['acc']

# 그래프로 표현
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Validation_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Training_loss')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='best')
# plt.axis([0, 20, 0, 0.35])
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')

In [ ]:
# 그래프로 표현
x_len = np.arange(len(y_acc))
plt.plot(x_len, y_vacc, marker='.', c="red", label='Validation_acc')
plt.plot(x_len, y_acc, marker='.', c="blue", label='Training_acc')

# 그래프에 그리드를 주고 레이블을 표시
plt.legend(loc='best')
# plt.axis([0, 20, 0, 0.35])
plt.grid()
plt.xlabel('epoch')